# Klasteryzacja ofert na podstawie tytułów

W dziejszych czasach nieodłącznym elementem sektora usług konsumenckich jest rekomendacja pewnych przedmiotów. Jedną z firm wykorzystujących tą technikę w świadczonych usługach jest Allegro.pl, największa platforma transakcyjna on-line w Polsce, o czym świadczy dostępna na ich stronie sekcja "polecane dla ciebie". Niestety czasem zdarza się, że system rekomendacyjny poleci użytkownikowi oferty przedstawiające ten sam produkt, ale o innym tytule. Aby uniknąć tego typu sytuacji można próbować łączyć aukcje z tymi samymi przedmiotami w pewne grupy. I właśnie ten projekt miał na celu zmierzenie się z problemem klasteryzacji ofert z portalu Allegro na podstawie ich tytułów. 

# Dane

## Pobranie danych

Dane do projektu należało pobrać przy pomocy Allegro WebAPI, tzn. usługi sieciowej, która umożliwia wymianę informacji między zasobami Allegro a oprogramowaniem zewnętrznym.

W naszym projekcie postanowiliśmy się skupić na książkach o fantastyce, co wynikało głównie ze zwięzłości wybranej kategorii i łatwej formy tytułów ofert. W tym celu pobraliśmy wszystkie możliwe (44357) tytuły dostępnych aukcji w dniu 9 czerwca 2017 roku.

In [10]:
import pandas as pd

data = pd.read_csv("titles_books.csv")
print("Liczba ofert: " + str(data.shape[0]))
data.head(10)

Liczba ofert: 44357


,Unnamed: 0,title
0,24118669,N.Stephenson USTRÓJ ŚWIATA- pomoc rottka.pl
1,24119018,T Canavan NOWICJUSZKA pomoc rottka.pl
2,24119019,T Canavan UCZENNICA MAGA pomoc rottka.pl
3,24119035,T Canavan UCZENNICA MAGA pomoc rottka.pl
4,24174585,ZAMEK ZŁUDZEŃ Mercedes Lackey Josepha Sherman
5,24176443,ZWIADOWCY WCZESNE LATA TURNIEJ W GORLANIE Flan...
6,24186459,*BLOX* Więzień na Marsie. G. Le Rouge. KIC
7,24189517,Brama Ivrel C.J. Cherryh
8,24194754,Silentium Universi - Dariusz Domagalski fantasy
9,24196169,NIEŚMIERTELNY Sharon Sala


## Czyszczenie danych

Zawsze podczas pracy z danymi, w szczególności rzeczywistymi, należy je wyczyścić na tyle na ile jest to możliwe.

Wykorzystana przez nas wstępna obróbka danych to:
1. Usunięcie wszystkich znaków interpunkcyjnych,
2. Zamiana inicjałów autorów na ciąg znaków (np. C.J. -> CJ),
3. Zamiana wielkich liter na małe,
4. Usunięcie pojedynczych znaków i liter,
5. Usunięcie stopwordsów (lista 3717 słów i symboli charakterystycznych dla ofert związanych z książkami),
6. Przedstawienie tytułów ofert jako wektory słów.

# Miary jakości

W przypadku gdy znane są prawdziwe klasy jest możliwe zdefiniowanie intuicyjnych miar oceny klasteryzacji:
1. homogeniczność - miara określająca czy każdy klaster zawiera tylko elementy z tej samej jednej klasy,
2. kompletność - miara określająca czy wszyscy członkowie jednej klasy należą do tego samego klastra,
3. V-miara - średnia harmoniczna powyższych miar.

Przedziałem wartości tych miar jest $[0,1]$, gdzie $0$ oznacza złe dopasowanie, a $1$ perfekcyjne. Główną zaletą tych miar jest ich intuicyjność oraz brak wymogów co do założeń struktury modelu do klasteryzacji.

Więcej o tych miarach i wzorach je reprezentujących można przeczytać [tutaj](http://scikit-learn.org/stable/modules/clustering.html#homogeneity-completeness-and-v-measure).
